### 层和块

In [1]:
# 多层感知机

import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)

net(X)

tensor([[-0.0043,  0.0149,  0.2440,  0.0671, -0.1666, -0.1139, -0.0839, -0.0426,
          0.0912,  0.2010],
        [ 0.1549,  0.0946,  0.3302,  0.1321, -0.1801, -0.0933, -0.0752, -0.0326,
          0.0088,  0.1871]], grad_fn=<AddmmBackward0>)

In [2]:
### 自定义块

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0968,  0.0048,  0.2492,  0.0842,  0.0803,  0.1312,  0.0904,  0.1101,
          0.0843, -0.0390],
        [-0.1671,  0.0337,  0.1752,  0.1254,  0.0886, -0.0249,  0.1034,  0.1226,
          0.0097, -0.0126]], grad_fn=<AddmmBackward0>)

### 顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0560,  0.1126,  0.1300,  0.1908,  0.0544, -0.0409, -0.0982,  0.2174,
          0.0595,  0.0620],
        [ 0.0057,  0.0848,  0.1740,  0.1631,  0.0601, -0.0477, -0.2266,  0.1988,
          0.0622,  0.1966]], grad_fn=<AddmmBackward0>)

In [6]:
### 在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0925, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法

In [8]:

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0486, grad_fn=<SumBackward0>)

### 参数管理

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)


tensor([[0.4600],
        [0.4471]], grad_fn=<AddmmBackward0>)


#### 参数访问

In [10]:
print(net[0].state_dict())
print(net[1].state_dict())
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1019, -0.2338, -0.4165, -0.4202],
        [-0.1343,  0.0165, -0.1415, -0.1900],
        [ 0.1175, -0.1750, -0.4781,  0.2651],
        [ 0.2124,  0.2053, -0.2927, -0.0477],
        [-0.4290,  0.2913,  0.0024, -0.1934],
        [-0.3933, -0.1298, -0.1690,  0.1641],
        [-0.3787,  0.3875, -0.1338,  0.0913],
        [ 0.2205, -0.2857,  0.0550,  0.3879]])), ('bias', tensor([ 0.4935, -0.2763,  0.3627, -0.0211, -0.1843, -0.4838,  0.4359,  0.3923]))])
OrderedDict()
OrderedDict([('weight', tensor([[-0.1685,  0.2054, -0.0010, -0.0396, -0.2803, -0.2439,  0.1820,  0.2508]])), ('bias', tensor([0.2294]))])


In [11]:
### 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2294], requires_grad=True)
tensor([0.2294])


In [12]:
net[2].weight.grad == None, net[ 2].weight.grad 

(True, None)

### 一次性访问所有参数

In [13]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


### 从嵌套块收集参数

In [14]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 内置初始化

In [15]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0045, -0.0073,  0.0047, -0.0056]), tensor(0.))

In [16]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
    
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

### 对某些块应用不同的初始化方法

In [17]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2812, -0.4900, -0.2171,  0.5971])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化


In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


tensor([[ 0.0000, -8.3972,  6.9154,  8.1943],
        [-9.5803,  0.0000,  0.0000, -5.3965]], grad_fn=<SliceBackward0>)

In [19]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.3972,  7.9154,  9.1943])

### 参数绑定

In [20]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 自定义层

In [21]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

### 将层作为逐渐合并到构建更复杂的模型中

In [22]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-4.3656e-09, grad_fn=<MeanBackward0>)

### 带参数的图层

In [23]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

denese = MyLinear(5, 3)
denese.weight

Parameter containing:
tensor([[-0.1286,  0.1520,  0.4583],
        [-0.6330, -0.5341,  0.7511],
        [-0.8845, -0.3332, -0.6369],
        [-0.4443, -0.9583, -0.2949],
        [-1.3239,  0.5056, -0.0208]], requires_grad=True)

In [24]:
### 使用自定义层直接执行正向传播计算

In [25]:
denese(torch.rand(2,5))

tensor([[0.0000, 1.4609, 1.1237],
        [0.0000, 1.9038, 1.6188]])

In [26]:
### 使用自定义层构建模型

In [27]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2,64))

tensor([[5.7828],
        [0.0000]])

### 读写文件

In [28]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])